In [15]:
from hmcode import BiHMCode_bispectrum
import pyccl as ccl
import numpy as np
from scipy.optimize import minimize

In [7]:
# Cosmology
Omega_c = 0.25
Omega_b = 0.05
h = 0.7
ns = 0.96
sigma_8  = 0.8

Cosmology=ccl.Cosmology(Omega_c=Omega_c, Omega_b=Omega_b, h=h, n_s=ns, sigma8=sigma_8)
ingredients={'hmf': 'Sheth & Tormen (1999)', 
             'concentration': 'Duffy et al. (2008)',
             'halo definition': 'Mvir', 
             'profile': 'NFW'}

In [8]:
# test data

freePars={'eta':0.139, 
                    'B':5.2, 
                    'alpha1':1, 
                    'alpha2':1,
                    'kstar':0.07,
                    'f':0.019,
                    'kd':0.073,
                    'nd':2.85}

kmin, kmax = 1e-3, 10.
nk = 51
ks = np.logspace(np.log10(kmin), np.log10(kmax), nk)

Mmin, Mmax = 1e9, 1e17
nM = 256
Ms = np.logspace(np.log10(Mmin), np.log10(Mmax), nM)

z = 0.
a=1/(1+z)

Bispec_data, _ , _, _ =BiHMCode_bispectrum(ks, Ms, Cosmology, z=z, 
                                            ingredients=ingredients,
                                            freePars=freePars,
                                            fastCalc=True,
                                            onlyEquilateral=False, dewiggle=True)

In [10]:
# Fit

cosmos=[Cosmology]
redshifts=[z]
datas=[Bispec_data]


In [ ]:

def chi2(cosmos, redshifts, pars, data):
    diff=0
    ix=0
    for cosmo in cosmos:
        for z in redshifts:
            data=data[ix]
            model, _, _, _ = BiHMCode_bispectrum(ks,Ms,cosmo, z=z, ingredients=ingredients,
                                                 freePars=pars,
                                                 fastCalc=True,
                                                 onlyEquilateral=False,
                                                 dewiggle=True)
            diff+=np.sum(np.abs(model-data))

    return diff

In [29]:
def chi2_wrapper(params, fixed):
    cosmos=fixed['cosmos']
    redshifts=fixed['redshifts']
    data=fixed['data']

    pars={'eta':params[0]*0.139, 
                    'B':params[1]*5.2, 
                    'alpha1':params[2], 
                    'alpha2':params[3],
                    'kstar':params[4]*0.07,
                    'f':params[5]*0.019,
                    'kd':params[6]*0.073,
                    'nd':params[7]*2.85}
    diff= chi2(cosmos, redshifts, pars, data)
    print(pars, diff)
    return diff

In [27]:
initial_guess=[0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7]

fixed={'cosmos': cosmos, 'redshifts': redshifts, 'data': datas}

In [30]:
result=minimize(chi2_wrapper, initial_guess, args=(fixed,))

{'eta': 0.0973, 'B': 3.6399999999999997, 'alpha1': 0.7, 'alpha2': 0.7, 'kstar': 0.049, 'f': 0.0133, 'kd': 0.05109999999999999, 'nd': 1.9949999999999999} 864261718692382.0
{'eta': 0.09730000207126141, 'B': 3.6399999999999997, 'alpha1': 0.7, 'alpha2': 0.7, 'kstar': 0.049, 'f': 0.0133, 'kd': 0.05109999999999999, 'nd': 1.9949999999999999} 864261718366250.2
{'eta': 0.0973, 'B': 3.640000077486038, 'alpha1': 0.7, 'alpha2': 0.7, 'kstar': 0.049, 'f': 0.0133, 'kd': 0.05109999999999999, 'nd': 1.9949999999999999} 864261723081075.6
{'eta': 0.0973, 'B': 3.6399999999999997, 'alpha1': 0.7000000149011611, 'alpha2': 0.7, 'kstar': 0.049, 'f': 0.0133, 'kd': 0.05109999999999999, 'nd': 1.9949999999999999} 864261715206512.8
{'eta': 0.0973, 'B': 3.6399999999999997, 'alpha1': 0.7, 'alpha2': 0.7000000149011611, 'kstar': 0.049, 'f': 0.0133, 'kd': 0.05109999999999999, 'nd': 1.9949999999999999} 864271334501994.2
{'eta': 0.0973, 'B': 3.6399999999999997, 'alpha1': 0.7, 'alpha2': 0.7, 'kstar': 0.04900000104308128, 'f